In [1]:
import pandas as pd
import glob
import time
import duckdb

In [2]:
conn = duckdb.connect()

In [3]:
df = conn.execute("""
    select * 
    from read_csv_auto('/Users/cliffordfrempong/Desktop/PersonalProjects/duckdb_tuts/dataset/*.csv', header=True)

""").df()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [5]:
df.isnull().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price Each          545
Order Date          545
Purchase Address    545
dtype: int64

In [6]:
df = df.dropna(how='all')

In [7]:
conn.execute("""
    select * from df
""").df()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,295665,Macbook Pro Laptop,1,1700,12/30/19 00:01,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.0,12/29/19 07:03,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,12/12/19 18:21,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,12/22/19 15:13,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,12/18/19 12:38,"43 Hill St, Atlanta, GA 30301"
...,...,...,...,...,...,...
186300,222905,AAA Batteries (4-pack),1,2.99,06/07/19 19:02,"795 Pine St, Boston, MA 02215"
186301,222906,27in FHD Monitor,1,149.99,06/01/19 19:29,"495 North St, New York City, NY 10001"
186302,222907,USB-C Charging Cable,1,11.95,06/22/19 18:57,"319 Ridge St, San Francisco, CA 94016"
186303,222908,USB-C Charging Cable,1,11.95,06/26/19 18:35,"916 Main St, San Francisco, CA 94016"


In [8]:
conn.execute("""
create or replace table sales as
    select 
        "Order ID"::integer as order_id,
        Product as product,
        "Quantity Ordered"::integer as quantity,
        "Price Each"::decimal as price,
        strptime("Order Date", '%m/%d/%Y %H:%M')::date as order_date,
        "Purchase Address" as purchase_address
    from df
    where try_cast("Order ID" as integer)notnull
""")

In [9]:
conn.execute("""
from sales
""").df()

,order_id,product,quantity,price,order_date,purchase_address
0,295665,Macbook Pro Laptop,1,1700.00,1773-08-27 22:43:41.128654848,"136 Church St, New York City, NY 10001"
1,295666,LG Washing Machine,1,600.00,1773-08-26 22:43:41.128654848,"562 2nd St, New York City, NY 10001"
2,295667,USB-C Charging Cable,1,11.95,1773-08-09 22:43:41.128654848,"277 Main St, New York City, NY 10001"
3,295668,27in FHD Monitor,1,149.99,1773-08-19 22:43:41.128654848,"410 6th St, San Francisco, CA 94016"
4,295669,USB-C Charging Cable,1,11.95,1773-08-15 22:43:41.128654848,"43 Hill St, Atlanta, GA 30301"
...,...,...,...,...,...,...
185945,222905,AAA Batteries (4-pack),1,2.99,1773-02-02 22:43:41.128654848,"795 Pine St, Boston, MA 02215"
185946,222906,27in FHD Monitor,1,149.99,1773-01-27 22:43:41.128654848,"495 North St, New York City, NY 10001"
185947,222907,USB-C Charging Cable,1,11.95,1773-02-17 22:43:41.128654848,"319 Ridge St, San Francisco, CA 94016"
185948,222908,USB-C Charging Cable,1,11.95,1773-02-21 22:43:41.128654848,"916 Main St, San Francisco, CA 94016"


In [10]:
conn.close()